# Log output

We use the same example as in the quickstart.

In [1]:
using MIPVerify
using Gurobi

In [2]:
mnist = MIPVerify.read_datasets("MNIST")
n1 = MIPVerify.get_example_network_params("MNIST.n1")
sample_image = MIPVerify.get_image(mnist.train.images, 1)
target_label_index = 9

9

## Viewing Log Output

Our package logs to STDOUT at the `debug`, `info`, `notice` and `warn` levels. 
The default output level is to show logs only at the `notice` level and above, which is also the recommended level.

In [3]:
d = MIPVerify.find_adversarial_example(n1, sample_image, target_label_index, GurobiSolver(), rebuild=true);

[notice | MIPVerify]: Rebuilding model from scratch. This may take some time as we determine upper and lower bounds for the input to each non-linear unit. The model built will be cached and re-used for future solves, unless you explicitly set rebuild=true.


  Calculating upper bounds: 100%|███████████████████████| Time: 0:00:02
  Calculating lower bounds: 100%|███████████████████████| Time: 0:00:00
  Imposing relu constraint: 100%|███████████████████████| Time: 0:00:00
  Calculating upper bounds: 100%|███████████████████████| Time: 0:00:01
  Calculating lower bounds: 100%|███████████████████████| Time: 0:00:00


[notice | MIPVerify]: Attempting to find adversarial example. Neural net predicted label is 8, target labels are [9]
Optimize a model with 3385 rows, 3256 columns and 71132 nonzeros
Variable types: 3196 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [2e-05, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+02]
  RHS range        [1e-02, 7e+02]
Presolve removed 2860 rows and 2184 columns
Presolve time: 0.13s
Presolved: 525 rows, 1072 columns, 65472 nonzeros

MIP start did not produce a new incumbent solution
MIP start violates constraint R784 by 70.892854817

Variable types: 1012 continuous, 60 integer (60 binary)

Root relaxation: objective 0.000000e+00, 241 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    9          -    0.00000      -     -    0s
Another try with MIP start
  

More information is available if we set the log level to `debug`.

In [4]:
MIPVerify.setloglevel!("debug")
d = MIPVerify.find_adversarial_example(n1, sample_image, target_label_index, GurobiSolver(), rebuild=true)

[notice | MIPVerify]: Rebuilding model from scratch. This may take some time as we determine upper and lower bounds for the input to each non-linear unit. The model built will be cached and re-used for future solves, unless you explicitly set rebuild=true.
[info | MIPVerify]: Applying Flatten() ... 
[info | MIPVerify]: Applying Linear(784 -> 40) ... 
[info | MIPVerify]: Applying ReLU() ...
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0
[debug | MIPVerify]:   Δu = 0.0

Dict{Symbol,Any} with 7 entries:
  :PerturbationParameters => additive
  :TargetIndexes          => [9]
  :SolveStatus            => :Optimal
  :Output                 => JuMP.GenericAffExpr{Float64,JuMP.Variable}[-0.0120…
  :Model                  => Minimization problem with:…
  :Perturbation           => JuMP.Variable[__anon__ __anon__ __anon__ __anon__ …
  :PerturbedInput         => JuMP.Variable[__anon__ __anon__ __anon__ __anon__ …

When we select `rebuild=false`, we do not do any of the work in determining upper and lower bounds, and none of the `info` and `debug` logs are produced.

In [5]:
d = MIPVerify.find_adversarial_example(n1, sample_image, target_label_index, GurobiSolver(), rebuild=false);

[notice | MIPVerify]: Loading model from cache.
[notice | MIPVerify]: Attempting to find adversarial example. Neural net predicted label is 8, target labels are [9]
Optimize a model with 3385 rows, 3256 columns and 71132 nonzeros
Variable types: 3196 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [2e-05, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+02]
  RHS range        [1e-02, 7e+02]
Presolve removed 2860 rows and 2184 columns
Presolve time: 0.14s
Presolved: 525 rows, 1072 columns, 65472 nonzeros

MIP start did not produce a new incumbent solution
MIP start violates constraint R784 by 70.892854817

Variable types: 1012 continuous, 60 integer (60 binary)

Root relaxation: objective 0.000000e+00, 241 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    9          -    0.00000 

## Handling log output

We use the logger provided by the [`Memento`](https://github.com/invenia/Memento.jl) package. Rather than just having logs written to STDOUT, you can choose to capture the output (for example, to write logs to file) by adding your own handlers to `MIPVerify.LOGGER`. See the Memento documentation for more.

In [6]:
typeof(MIPVerify.LOGGER)

Memento.Logger